In [1]:
import socket
import threading
from concurrent.futures import ThreadPoolExecutor

def scan_port(ip, port, timeout=1):
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(timeout)
            result = s.connect_ex((ip, port))
            if result == 0:
                try:
                    service = socket.getservbyport(port)
                except:
                    service = "unknown"
                print(f"[+] Порт {port} ({service}) открыт")
                return port
    except Exception as e:
        pass
    return None

def port_scanner(ip, ports_range):
    open_ports = []
    with ThreadPoolExecutor(max_workers=100) as executor:
        results = executor.map(lambda port: scan_port(ip, port), ports_range)
        for port in results:
            if port:
                open_ports.append(port)
    return open_ports

# Пример использования
target = "example.com"
ports = range(1, 1025)  # Сканируем первые 1024 порта
open_ports = port_scanner(target, ports)
print(f"Открытые порты: {open_ports}")

[+] Порт 80 (unknown) открыт
[+] Порт 443 (unknown) открыт
Открытые порты: [80, 443]


In [2]:
import requests

def check_vulnerable_service(ip, port):
    try:
        # Пример: проверка HTTP-сервера
        if port == 80 or port == 443:
            url = f"http://{ip}:{port}" if port == 80 else f"https://{ip}:{port}"
            response = requests.get(url, timeout=3, verify=False)
            server_header = response.headers.get('Server', '')

            # Проверка уязвимых версий nginx
            if "nginx/1.18.0" in server_header:
                print(f"[!] Уязвимая версия nginx обнаружена на порту {port}")

    except requests.exceptions.RequestException:
        pass

# Запуск для всех открытых портов
for port in open_ports:
    check_vulnerable_service(target, port)

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'example.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
def check_sql_injection(url):
    test_payloads = ["'", "\"", "'; --", "1' OR '1'='1"]
    vulnerable = False

    for payload in test_payloads:
        test_url = f"{url}?id={payload}"
        try:
            response = requests.get(test_url, timeout=3)
            if "sql" in response.text.lower() or "syntax" in response.text.lower():
                print(f"[!] Возможна SQLi: {test_url}")
                vulnerable = True
        except:
            continue

    return vulnerable

# Пример использования
check_sql_injection("http://test-site.com/page")

False

In [4]:
def check_xss(url):
    xss_payloads = [
        "<script>alert('XSS')</script>",
        "\"><script>alert('XSS')</script>",
        "javascript:alert('XSS')"
    ]

    for payload in xss_payloads:
        test_url = f"{url}?search={payload}"
        try:
            response = requests.get(test_url, timeout=3)
            if payload in response.text:
                print(f"[!] Возможна XSS: {test_url}")
        except:
            continue

check_xss("http://test-site.com/search")

In [5]:
import requests
import json

def check_cve(service_name, version):
    # Используем API NVD (National Vulnerability Database)
    url = f"https://services.nvd.nist.gov/rest/json/cves/1.0"
    params = {
        "keyword": f"{service_name} {version}",
        "resultsPerPage": 10
    }

    try:
        response = requests.get(url, params=params, timeout=5)
        data = response.json()

        if data["totalResults"] > 0:
            print(f"[!] Найдены уязвимости для {service_name} {version}:")
            for item in data["result"]["CVE_Items"][:3]:
                cve_id = item["cve"]["CVE_data_meta"]["ID"]
                description = item["cve"]["description"]["description_data"][0]["value"]
                print(f"    {cve_id}: {description[:100]}...")
    except Exception as e:
        print(f"Ошибка при проверке CVE: {e}")

# Пример использования
check_cve("nginx", "1.18.0")

Ошибка при проверке CVE: Expecting value: line 1 column 1 (char 0)


In [6]:
def main():
    target = input("Введите цель (IP или домен): ")

    print("[*] Сканирование портов...")
    ports = range(1, 1001)  # Первые 1000 портов
    open_ports = port_scanner(target, ports)

    print("[*] Проверка сервисов...")
    for port in open_ports:
        check_vulnerable_service(target, port)

    print("[*] Проверка веб-уязвимостей...")
    if 80 in open_ports:
        check_sql_injection(f"http://{target}")
        check_xss(f"http://{target}")
    if 443 in open_ports:
        check_sql_injection(f"https://{target}")
        check_xss(f"https://{target}")

if __name__ == "__main__":
    main()

Введите цель (IP или домен): google.com
[*] Сканирование портов...
[+] Порт 80 (unknown) открыт
[+] Порт 443 (unknown) открыт
[*] Проверка сервисов...
[*] Проверка веб-уязвимостей...
[!] Возможна SQLi: http://google.com?id='


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[!] Возможна SQLi: http://google.com?id="
[!] Возможна SQLi: http://google.com?id='; --
[!] Возможна SQLi: http://google.com?id=1' OR '1'='1
[!] Возможна SQLi: https://google.com?id='
[!] Возможна SQLi: https://google.com?id="
[!] Возможна SQLi: https://google.com?id='; --
[!] Возможна SQLi: https://google.com?id=1' OR '1'='1
